In [2]:
from pyspark.sql import functions as f
from pyspark.sql import SparkSession
import os
import pandas as pd
import random

In [3]:
spark = SparkSession.builder.getOrCreate()

In [4]:
spark_fp = os.path.join("/", "Volumes", "Marceline Jr.", "Spotify Dataset", "sampled_users.csv")
spark_fp

'/Volumes/Marceline Jr./Spotify Dataset/sampled_users.csv'

In [5]:
df = spark.read.option("header", "true").csv(spark_fp)

In [6]:
users = df.toPandas()

In [7]:
tf_path_one = os.path.join("/", "Volumes", "Marceline Jr.", "Spotify Dataset", "track_features", "tf_000000000000.csv")
tf_path_two = os.path.join("/", "Volumes", "Marceline Jr.", "Spotify Dataset", "track_features", "tf_000000000001.csv")

In [8]:
track_features_one = pd.read_csv(tf_path_one)
track_features_two = pd.read_csv(tf_path_two)

In [9]:
track_features = pd.concat([track_features_one, track_features_two])

In [10]:
userFeatures = pd.merge(users, track_features, left_on = 'track_id_clean', right_on = 'track_id')
nonModifiedFeatures = pd.merge(users, track_features, left_on = 'track_id_clean', right_on = 'track_id')

In [11]:
pd.set_option('display.max_columns', None)

In [12]:
userFeatures.head()

,session_id,session_position,session_length,track_id_clean,skip_1,skip_2,skip_3,not_skipped,context_switch,no_pause_before_play,short_pause_before_play,long_pause_before_play,hist_user_behavior_n_seekfwd,hist_user_behavior_n_seekback,hist_user_behavior_is_shuffle,hour_of_day,date,premium,context_type,hist_user_behavior_reason_start,hist_user_behavior_reason_end,track_id,duration,release_year,us_popularity_estimate,acousticness,beat_strength,bounciness,danceability,dyn_range_mean,energy,flatness,instrumentalness,key,liveness,loudness,mechanism,mode,organism,speechiness,tempo,time_signature,valence,acoustic_vector_0,acoustic_vector_1,acoustic_vector_2,acoustic_vector_3,acoustic_vector_4,acoustic_vector_5,acoustic_vector_6,acoustic_vector_7
0,42_50cb77d0-9f69-4948-98a6-445f4f1d98da,1,18,t_3bc02be3-ab02-4c9f-94af-04910b00a14c,true,true,true,false,0,0,0,0,0,0,true,16,2018-08-26,true,user_collection,fwdbtn,fwdbtn,t_3bc02be3-ab02-4c9f-94af-04910b00a14c,284.506683,2013,99.971866,0.279228,0.505583,0.565925,0.624569,8.799324,0.641628,0.983994,1.215048e-06,0,0.103448,-7.291,0.250729,major,0.565409,0.168414,84.543999,4,0.332708,-0.747017,0.236945,0.157640,0.053084,-0.247379,-0.015617,-0.557144,0.094659
1,42_50cb77d0-9f69-4948-98a6-445f4f1d98da,2,18,t_529537c9-bfb0-4058-bb9a-1d93ef071263,true,true,true,false,0,1,0,0,0,0,true,16,2018-08-26,true,user_collection,fwdbtn,fwdbtn,t_529537c9-bfb0-4058-bb9a-1d93ef071263,258.399994,2013,99.978626,0.174235,0.539210,0.580000,0.740413,8.774446,0.474385,1.037253,8.307667e-06,11,0.152949,-8.708,0.538653,minor,0.348711,0.062201,107.454002,4,0.081849,-0.859858,0.301731,0.208637,0.108421,-0.310408,-0.036791,-0.545892,0.162957
2,42_50cb77d0-9f69-4948-98a6-445f4f1d98da,3,18,t_c29129f2-7b64-4c1e-9587-676632b7800d,true,true,true,false,0,0,1,1,0,0,true,16,2018-08-26,true,user_collection,fwdbtn,fwdbtn,t_c29129f2-7b64-4c1e-9587-676632b7800d,875.306641,2014,99.978268,0.854981,0.560593,0.650291,0.651595,10.094535,0.455094,1.013489,5.354104e-10,8,0.171293,-8.673,0.200926,major,0.827500,0.115338,83.151001,4,0.325166,-0.517902,0.006190,0.087467,0.191870,-0.131095,-0.131147,-0.435564,-0.100854
3,42_50cb77d0-9f69-4948-98a6-445f4f1d98da,4,18,t_6c1ce50d-d33a-4502-8c58-335f23b03145,true,true,true,false,0,1,0,0,0,0,true,16,2018-08-26,true,user_collection,fwdbtn,fwdbtn,t_6c1ce50d-d33a-4502-8c58-335f23b03145,237.946671,2011,99.934497,0.092926,0.522707,0.645280,0.556230,10.356951,0.573947,1.029153,2.194449e-04,10,0.113326,-8.555,0.464015,minor,0.384652,0.095105,138.953995,4,0.290325,-0.854975,0.354534,0.233016,0.039705,-0.422565,0.041233,-0.445123,0.352062
4,42_50cb77d0-9f69-4948-98a6-445f4f1d98da,5,18,t_85418d4c-2b2d-4270-836e-ef789bb28555,true,true,true,false,0,1,0,0,0,0,true,16,2018-08-26,true,user_collection,fwdbtn,fwdbtn,t_85418d4c-2b2d-4270-836e-ef789bb28555,223.216324,2013,99.836948,0.259199,0.505158,0.562672,0.634538,8.739496,0.702552,1.020529,4.562811e-01,1,0.113926,-6.786,0.263538,major,0.552069,0.156347,85.946999,4,0.425356,-0.891446,0.229755,0.256360,0.193858,-0.322325,-0.040302,-0.553673,0.162334


In [13]:
cols = list(userFeatures.columns)

In [14]:
drop = cols[1:25]

In [15]:
userFeatures.drop(columns = drop, inplace = True)

In [16]:
userFeatures['mode'] = userFeatures['mode'].apply(lambda x: 1 if x == 'major' else 0)

In [17]:
features = userFeatures.groupby('session_id').mean()

In [18]:
X = features.reset_index().drop('session_id', axis = 1)
#we wanted to groupby so we would cluster by user avg song features

Drop our code so we only have the track features for clustering

In [19]:
from sklearn.cluster import KMeans
cluster = KMeans(n_clusters = 3)
cluster.fit(X)

In [22]:
cluster.labels_

array([2, 0, 1, 2, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 2, 1, 2, 0, 0, 0, 1,
       0, 1, 0, 0, 2, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 1, 2, 2,
       0, 0, 1, 0, 2, 0, 1, 1, 2, 0, 0, 0, 0, 0, 0, 1, 1, 2, 0, 1, 0, 1,
       0, 1, 0, 1, 1, 1, 1, 2, 0, 2, 1, 0, 1, 2, 0, 0, 0, 2, 1, 1, 0, 0,
       0, 2, 2, 0, 2, 0, 0, 1, 1, 2, 0, 2], dtype=int32)

grab our cluster labels and re-add them to our features so we have session_id and corresponding cluster

In [23]:
y = userFeatures.groupby('session_id').mean()

In [24]:
y['Cluster'] = cluster.labels_

In [25]:
userOne = y[y['Cluster'] == 0]
userTwo = y[y['Cluster'] == 1]
userThree = y[y['Cluster'] == 2]

In [26]:
userOneFeatures = userOne.merge(nonModifiedFeatures, on = 'session_id', how = 'inner')
userTwoFeatures = userTwo.merge(nonModifiedFeatures, on = 'session_id', how = 'inner')
userThreeFeatures = userThree.merge(nonModifiedFeatures, on = 'session_id', how = 'inner')

In [27]:
userOneFeatures.drop(['acousticness_x', 'beat_strength_x', 'bounciness_x',
       'danceability_x', 'dyn_range_mean_x', 'energy_x', 'flatness_x',
       'instrumentalness_x', 'key_x', 'liveness_x', 'loudness_x',
       'mechanism_x', 'mode_x', 'organism_x', 'speechiness_x', 'tempo_x',
       'time_signature_x', 'valence_x', 'acoustic_vector_0_x',
       'acoustic_vector_1_x', 'acoustic_vector_2_x', 'acoustic_vector_3_x',
       'acoustic_vector_4_x', 'acoustic_vector_5_x', 'acoustic_vector_6_x',
       'acoustic_vector_7_x', 'Cluster'], axis = 1, inplace = True)
userTwoFeatures.drop(['acousticness_x', 'beat_strength_x', 'bounciness_x',
       'danceability_x', 'dyn_range_mean_x', 'energy_x', 'flatness_x',
       'instrumentalness_x', 'key_x', 'liveness_x', 'loudness_x',
       'mechanism_x', 'mode_x', 'organism_x', 'speechiness_x', 'tempo_x',
       'time_signature_x', 'valence_x', 'acoustic_vector_0_x',
       'acoustic_vector_1_x', 'acoustic_vector_2_x', 'acoustic_vector_3_x',
       'acoustic_vector_4_x', 'acoustic_vector_5_x', 'acoustic_vector_6_x',
       'acoustic_vector_7_x', 'Cluster'], axis = 1, inplace = True)
userThreeFeatures.drop(['acousticness_x', 'beat_strength_x', 'bounciness_x',
       'danceability_x', 'dyn_range_mean_x', 'energy_x', 'flatness_x',
       'instrumentalness_x', 'key_x', 'liveness_x', 'loudness_x',
       'mechanism_x', 'mode_x', 'organism_x', 'speechiness_x', 'tempo_x',
       'time_signature_x', 'valence_x', 'acoustic_vector_0_x',
       'acoustic_vector_1_x', 'acoustic_vector_2_x', 'acoustic_vector_3_x',
       'acoustic_vector_4_x', 'acoustic_vector_5_x', 'acoustic_vector_6_x',
       'acoustic_vector_7_x', 'Cluster'], axis = 1, inplace = True)

In [28]:
userOneFeatures.rename(columns = lambda x: x[:-2] if x[-2:] == '_y' else x, inplace = True)
userTwoFeatures.rename(columns = lambda x: x[:-2] if x[-2:] == '_y' else x, inplace = True)
userThreeFeatures.rename(columns = lambda x: x[:-2] if x[-2:] == '_y' else x, inplace = True)

In [29]:
userOneFeatures['not_skipped'] = userOneFeatures['not_skipped'].apply(lambda x: 1 if x == True else 0)
userTwoFeatures['not_skipped'] = userTwoFeatures['not_skipped'].apply(lambda x: 1 if x == True else 0)
userThreeFeatures['not_skipped'] = userThreeFeatures['not_skipped'].apply(lambda x: 1 if x == True else 0)

In [30]:
userOneFeatures['premium']= userOneFeatures['premium'].apply(lambda x: 1 if x is True else 0)
userOneFeatures['hist_user_behavior_is_shuffle'] = userOneFeatures['hist_user_behavior_is_shuffle'].apply(lambda x: 1 if x is True else 0)

userTwoFeatures['premium']= userTwoFeatures['premium'].apply(lambda x: 1 if x is True else 0)
userTwoFeatures['hist_user_behavior_is_shuffle'] = userTwoFeatures['hist_user_behavior_is_shuffle'].apply(lambda x: 1 if x is True else 0)

userThreeFeatures['premium']= userThreeFeatures['premium'].apply(lambda x: 1 if x is True else 0)
userThreeFeatures['hist_user_behavior_is_shuffle'] = userThreeFeatures['hist_user_behavior_is_shuffle'].apply(lambda x: 1 if x is True else 0)

In [31]:
userOneFeatures.drop(['track_id_clean', 
         'skip_1', 
         'skip_2', 
         'skip_3',
         'hist_user_behavior_reason_end', 
         'session_id', #session id would bring our sklearn model down bc its not just a single value
         #different users don't consistently listen to the same amount of songs, vectors could be differing size
         #listen to different types of songs
         #session_id does not encode that information, it tells me nothing about my user's behavior even thought it is a numrical value
         #session_id points to the group of songs that the user listens to
         'track_id',
         'date'], 
        axis = 1, inplace = True)

In [32]:
userTwoFeatures.drop(['track_id_clean', 
         'skip_1', 
         'skip_2', 
         'skip_3',
         'hist_user_behavior_reason_end', 
         'session_id', #session id would bring our sklearn model down bc its not just a single value
         #different users don't consistently listen to the same amount of songs, vectors could be differing size
         #listen to different types of songs
         #session_id does not encode that information, it tells me nothing about my user's behavior even thought it is a numrical value
         #session_id points to the group of songs that the user listens to
         'track_id',
         'date'], 
        axis = 1, inplace = True)

In [33]:
userThreeFeatures.drop(['track_id_clean', 
         'skip_1', 
         'skip_2', 
         'skip_3',
         'hist_user_behavior_reason_end', 
         'session_id', #session id would bring our sklearn model down bc its not just a single value
         #different users don't consistently listen to the same amount of songs, vectors could be differing size
         #listen to different types of songs
         #session_id does not encode that information, it tells me nothing about my user's behavior even thought it is a numrical value
         #session_id points to the group of songs that the user listens to
         'track_id',
         'date'], 
        axis = 1, inplace = True)

In [35]:
userOneFeatures.shape

(800, 43)

In [ ]:
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier

In [ ]:
def runModel(features):
    as_is = ['session_position', 'session_length','hist_user_behavior_is_shuffle',
       'hour_of_day','premium','duration', 
       'release_year', 'us_popularity_estimate', 'acousticness',
       'beat_strength', 'bounciness', 'danceability', 'dyn_range_mean',
       'energy', 'flatness', 'instrumentalness', 'liveness', 'loudness',
       'mechanism', 'key', 'organism', 'speechiness', 'tempo',
       'time_signature', 'valence', 'acoustic_vector_0', 'acoustic_vector_1',
       'acoustic_vector_2', 'acoustic_vector_3', 'acoustic_vector_4',
       'acoustic_vector_5', 'acoustic_vector_6', 'acoustic_vector_7', 'context_switch', 'no_pause_before_play', 'short_pause_before_play',
       'long_pause_before_play', 'hist_user_behavior_n_seekfwd',
       'hist_user_behavior_n_seekback']
    ohe = ['mode','context_type', 'hist_user_behavior_reason_start']
    
    preproc = ColumnTransformer(
    transformers = [
        ('as_is', FunctionTransformer(lambda x: x), as_is),
        ('one_hot', OneHotEncoder(handle_unknown = 'ignore'), ohe),
    ]
)
    pl = Pipeline(steps = [('preprocessor', preproc), ('classifier', DecisionTreeClassifier(max_depth = 10))])
    x_train, x_test, y_train, y_test = train_test_split(features.drop('not_skipped', axis = 1), features['not_skipped'], test_size= 0.1)
    pl.fit(x_train, y_train)
    
    return pl.score(x_test,y_test)

- 100% accuracy issues: make sure we're not evaluating on the test set, more data(?)
- could it be predicitng the most popular song, song that is so popular that nobody skips it
- k-nearest neighbors
- jaccard vs cosine (if vectors are [0, 1] I like vs don't like, take dot product = number of similarites) jaccard is for membership

In [ ]:
runModel(userOneFeatures)

In [ ]:
runModel(userTwoFeatures)

In [ ]:
runModel(userThreeFeatures)

In [247]:
# #if we wanted to get nearest centroid
# from sklearn.neighbors.nearest_centroid import NearestCentroid
# clf = NearestCentroid()
# clf.fit(features, userFeatures['session_id'].unique())
# clf.centroids_